# Sign Language Digits Classifier
Using MobileNet transfer learning to classify ASL digits (0-9)

## 1. Import Dependencies

In [56]:
# Import dependencies
import os
import shutil
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.applications.mobilenet import preprocess_input

# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [57]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("Built with CUDA:", tf.test.is_built_with_cuda())

TensorFlow version: 2.10.1
Built with CUDA: True


In [58]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

Num GPUs Available:  1


## 2. Set Project Constants

In [59]:
# Project constants
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 30
VALIDATION_SPLIT = 0.2
NUM_CLASSES = 10

# Paths
DATASET_PATH = 'Sign-Language-Digits-Dataset/Dataset'
TRAIN_PATH = 'Sign-Language-Digits-Dataset/training'
TEST_PATH = 'Sign-Language-Digits-Dataset/test'

## 3. Create Dataset Splits
Creating training/ folder (85% of dataset) and test/ folder (15% of dataset)

In [60]:
# Create training/ and test/ folders with 85/15 split
def create_train_test_split(source_path, train_path, test_path, test_split=0.15):
    """Split dataset into training (85%) and test (15%) folders"""
    
    # Remove destination folders if they exist
    for path in [train_path, test_path]:
        if os.path.exists(path):
            shutil.rmtree(path)
        os.makedirs(path)
    
    total_train_images = 0
    total_test_images = 0
    
    # Process each digit (0-9)
    for digit in range(10):
        digit_source = os.path.join(source_path, str(digit))
        digit_train = os.path.join(train_path, str(digit))
        digit_test = os.path.join(test_path, str(digit))
        
        os.makedirs(digit_train)
        os.makedirs(digit_test)
        
        # Get all images for this digit
        all_images = [f for f in os.listdir(digit_source) if f.endswith('.JPG')]
        
        # Shuffle images
        random.shuffle(all_images)
        
        # Calculate split point (15% for test, 85% for training)
        num_test = int(len(all_images) * test_split)
        num_train = len(all_images) - num_test
        
        # Split images
        test_images = all_images[:num_test]
        train_images = all_images[num_test:]
        
        # Copy test images
        for img in test_images:
            src = os.path.join(digit_source, img)
            dst = os.path.join(digit_test, img)
            shutil.copy2(src, dst)
        
        # Copy training images
        for img in train_images:
            src = os.path.join(digit_source, img)
            dst = os.path.join(digit_train, img)
            shutil.copy2(src, dst)
        
        total_train_images += num_train
        total_test_images += num_test
        
        print(f'Digit {digit}: {num_train} training, {num_test} test')
    
    return total_train_images, total_test_images

print('Creating training/test split...')
num_train, num_test = create_train_test_split(DATASET_PATH, TRAIN_PATH, TEST_PATH)

print(f'\nTotal training images: {num_train} ({num_train/(num_train+num_test)*100:.1f}%)')
print(f'Total test images: {num_test} ({num_test/(num_train+num_test)*100:.1f}%)')

Creating training/test split...
Digit 0: 175 training, 30 test
Digit 1: 176 training, 30 test
Digit 2: 176 training, 30 test
Digit 3: 176 training, 30 test
Digit 4: 176 training, 31 test
Digit 5: 176 training, 31 test
Digit 6: 176 training, 31 test
Digit 7: 176 training, 30 test
Digit 8: 177 training, 31 test
Digit 9: 174 training, 30 test

Total training images: 1758 (85.3%)
Total test images: 304 (14.7%)


## 4. Create Data Generators
Setting up training and validation generators with 80/20 split

In [61]:
# Create image data generators with data augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=VALIDATION_SPLIT,
    # Data augmentation parameters
    rotation_range=15,              # Randomly rotate images by up to 15 degrees
    zoom_range=0.15,                # Randomly zoom in/out by up to 15%
    brightness_range=[0.8, 1.2],    # Randomly adjust brightness
    fill_mode='nearest'             # Fill pixels after transformations
)

# Validation generator (no augmentation, only rescaling)
val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=VALIDATION_SPLIT
)

# Training generator
train_generator = train_datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=42
)

# Validation generator
validation_generator = val_datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=True,
    seed=42
)

print(f'\nTraining samples: {train_generator.samples}')
print(f'Validation samples: {validation_generator.samples}')

Found 1409 images belonging to 10 classes.
Found 349 images belonging to 10 classes.

Training samples: 1409
Validation samples: 349


## 5. Build Model Architecture
Using MobileNetV2 as base model with frozen weights, adding custom classification head

In [62]:
# Import MobileNet and create model
print('Loading MobileNetV2...')
base_model = MobileNetV2(
    include_top=True, 
    weights='imagenet'
)

# for layer in base_model.layers[-30:]:
#     print(f"Layer: {layer.name}")
#     print(f"  Type: {layer.__class__.__name__}")
#     print(f"  Output Shape: {layer.output.shape}")
#     print(f"  Params: {layer.count_params()}")
#     print("-" * 50)

base_model.summary()

Loading MobileNetV2...
Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_12 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_12[0][0]']               
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                       

In [63]:
# Import MobileNet and create model
print('Loading MobileNetV2...')
base_model = MobileNetV2(
    include_top=True, 
    weights='imagenet'
)

for layer in base_model.layers[-20:]:
    print(f"Layer: {layer.name}")
    print(f"  Type: {layer.__class__.__name__}")
    print(f"  Output Shape: {layer.output.shape}")
    print(f"  Params: {layer.count_params()}")
    print("-" * 50)

Loading MobileNetV2...


Layer: block_15_expand_relu
  Type: ReLU
  Output Shape: (None, 7, 7, 960)
  Params: 0
--------------------------------------------------
Layer: block_15_depthwise
  Type: DepthwiseConv2D
  Output Shape: (None, 7, 7, 960)
  Params: 8640
--------------------------------------------------
Layer: block_15_depthwise_BN
  Type: BatchNormalization
  Output Shape: (None, 7, 7, 960)
  Params: 3840
--------------------------------------------------
Layer: block_15_depthwise_relu
  Type: ReLU
  Output Shape: (None, 7, 7, 960)
  Params: 0
--------------------------------------------------
Layer: block_15_project
  Type: Conv2D
  Output Shape: (None, 7, 7, 160)
  Params: 153600
--------------------------------------------------
Layer: block_15_project_BN
  Type: BatchNormalization
  Output Shape: (None, 7, 7, 160)
  Params: 640
--------------------------------------------------
Layer: block_15_add
  Type: Add
  Output Shape: (None, 7, 7, 160)
  Params: 0
-------------------------------------------

In [64]:
# base_model.trainable = False

# Build model on top of MobileNet
def get_predictions_layer(base_model_output):
    x = GlobalAveragePooling2D()(base_model_output)
    predictions = Dense(
        NUM_CLASSES,
        activation='softmax',
        # kernel_regularizer=regularizers.l2(0.1)
    )(x)
    return predictions

x = base_model.get_layer("out_relu").output
predictions = get_predictions_layer(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in model.layers[:-1]:
    layer.trainable = False

# Compile model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print('\nModel architecture:')
for layer in model.layers[-20:]:
    print(f"Layer: {layer.name}")
    print(f"  Type: {layer.__class__.__name__}")
    print(f"  Output Shape: {layer.output.shape}")
    print(f"  Params: {layer.count_params()}")
    print(f" Trainable: {layer.trainable}")
    print("-" * 50)


Model architecture:
Layer: block_15_expand_relu
  Type: ReLU
  Output Shape: (None, 7, 7, 960)
  Params: 0
 Trainable: False
--------------------------------------------------
Layer: block_15_depthwise
  Type: DepthwiseConv2D
  Output Shape: (None, 7, 7, 960)
  Params: 8640
 Trainable: False
--------------------------------------------------
Layer: block_15_depthwise_BN
  Type: BatchNormalization
  Output Shape: (None, 7, 7, 960)
  Params: 3840
 Trainable: False
--------------------------------------------------
Layer: block_15_depthwise_relu
  Type: ReLU
  Output Shape: (None, 7, 7, 960)
  Params: 0
 Trainable: False
--------------------------------------------------
Layer: block_15_project
  Type: Conv2D
  Output Shape: (None, 7, 7, 160)
  Params: 153600
 Trainable: False
--------------------------------------------------
Layer: block_15_project_BN
  Type: BatchNormalization
  Output Shape: (None, 7, 7, 160)
  Params: 640
 Trainable: False
-------------------------------------------

## 6. Train Model

In [65]:
# Train the model
print('Training model...')
callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
]
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1
)

Training model...
Epoch 1/30
45/45 [==============================] - 26s 534ms/step - loss: 1.4617 - accuracy: 0.5188 - val_loss: 1.1626 - val_accuracy: 0.5903 - lr: 0.0010
Epoch 2/30
45/45 [==============================] - 11s 249ms/step - loss: 0.7479 - accuracy: 0.7970 - val_loss: 0.9570 - val_accuracy: 0.6734 - lr: 0.0010
Epoch 3/30
45/45 [==============================] - 11s 249ms/step - loss: 0.5975 - accuracy: 0.8119 - val_loss: 0.7597 - val_accuracy: 0.7622 - lr: 0.0010
Epoch 4/30
45/45 [==============================] - 11s 253ms/step - loss: 0.4881 - accuracy: 0.8531 - val_loss: 0.7331 - val_accuracy: 0.7736 - lr: 0.0010
Epoch 5/30
45/45 [==============================] - 11s 254ms/step - loss: 0.4061 - accuracy: 0.8907 - val_loss: 0.6866 - val_accuracy: 0.7880 - lr: 0.0010
Epoch 6/30
45/45 [==============================] - 11s 248ms/step - loss: 0.3640 - accuracy: 0.9042 - val_loss: 0.6776 - val_accuracy: 0.7851 - lr: 0.0010
Epoch 7/30
45/45 [============================

In [73]:
x = base_model.get_layer("block_16_project_BN").output
predictions = get_predictions_layer(x)
model2 = Model(inputs=base_model.input, outputs=predictions)

for layer in model2.layers[:-1]:
    layer.trainable = False

model2.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print('\nModel architecture:')
for layer in model2.layers[-20:]:
    print(f"Layer: {layer.name}")
    print(f"  Type: {layer.__class__.__name__}")
    print(f"  Output Shape: {layer.output.shape}")
    print(f"  Params: {layer.count_params()}")
    print(f" Trainable: {layer.trainable}")
    print("-" * 50)


Model architecture:
Layer: block_14_add
  Type: Add
  Output Shape: (None, 7, 7, 160)
  Params: 0
 Trainable: False
--------------------------------------------------
Layer: block_15_expand
  Type: Conv2D
  Output Shape: (None, 7, 7, 960)
  Params: 153600
 Trainable: False
--------------------------------------------------
Layer: block_15_expand_BN
  Type: BatchNormalization
  Output Shape: (None, 7, 7, 960)
  Params: 3840
 Trainable: False
--------------------------------------------------
Layer: block_15_expand_relu
  Type: ReLU
  Output Shape: (None, 7, 7, 960)
  Params: 0
 Trainable: False
--------------------------------------------------
Layer: block_15_depthwise
  Type: DepthwiseConv2D
  Output Shape: (None, 7, 7, 960)
  Params: 8640
 Trainable: False
--------------------------------------------------
Layer: block_15_depthwise_BN
  Type: BatchNormalization
  Output Shape: (None, 7, 7, 960)
  Params: 3840
 Trainable: False
--------------------------------------------------
Layer

In [74]:
history = model2.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/30
45/45 [==============================] - 14s 267ms/step - loss: 2.1580 - accuracy: 0.2441 - val_loss: 1.7951 - val_accuracy: 0.3897 - lr: 0.0010
Epoch 2/30
45/45 [==============================] - 11s 253ms/step - loss: 1.3090 - accuracy: 0.5947 - val_loss: 1.3737 - val_accuracy: 0.5043 - lr: 0.0010
Epoch 3/30
45/45 [==============================] - 11s 252ms/step - loss: 0.9801 - accuracy: 0.7189 - val_loss: 1.1306 - val_accuracy: 0.6074 - lr: 0.0010
Epoch 4/30
45/45 [==============================] - 11s 252ms/step - loss: 0.7965 - accuracy: 0.7793 - val_loss: 1.0088 - val_accuracy: 0.6476 - lr: 0.0010
Epoch 5/30
45/45 [==============================] - 11s 250ms/step - loss: 0.6796 - accuracy: 0.8219 - val_loss: 0.9111 - val_accuracy: 0.6991 - lr: 0.0010
Epoch 6/30
45/45 [==============================] - 11s 250ms/step - loss: 0.5915 - accuracy: 0.8538 - val_loss: 0.8373 - val_accuracy: 0.7307 - lr: 0.0010
Epoch 7/30
45/45 [==============================] - 11s 250ms/st

In [68]:
x = base_model.get_layer("block_15_add").output
predictions = get_predictions_layer(x)
model3 = Model(inputs=base_model.input, outputs=predictions)

for layer in model3.layers[:-1]:
    layer.trainable = False

model3.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print('\nModel architecture:')
for layer in model3.layers[-20:]:
    print(f"Layer: {layer.name}")
    print(f"  Type: {layer.__class__.__name__}")
    print(f"  Output Shape: {layer.output.shape}")
    print(f"  Params: {layer.count_params()}")
    print(f" Trainable: {layer.trainable}")
    print("-" * 50)


Model architecture:
Layer: block_14_expand
  Type: Conv2D
  Output Shape: (None, 7, 7, 960)
  Params: 153600
 Trainable: False
--------------------------------------------------
Layer: block_14_expand_BN
  Type: BatchNormalization
  Output Shape: (None, 7, 7, 960)
  Params: 3840
 Trainable: False
--------------------------------------------------
Layer: block_14_expand_relu
  Type: ReLU
  Output Shape: (None, 7, 7, 960)
  Params: 0
 Trainable: False
--------------------------------------------------
Layer: block_14_depthwise
  Type: DepthwiseConv2D
  Output Shape: (None, 7, 7, 960)
  Params: 8640
 Trainable: False
--------------------------------------------------
Layer: block_14_depthwise_BN
  Type: BatchNormalization
  Output Shape: (None, 7, 7, 960)
  Params: 3840
 Trainable: False
--------------------------------------------------
Layer: block_14_depthwise_relu
  Type: ReLU
  Output Shape: (None, 7, 7, 960)
  Params: 0
 Trainable: False
--------------------------------------------

In [72]:
history = model3.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/30
45/45 [==============================] - 13s 260ms/step - loss: 2.3521 - accuracy: 0.1923 - val_loss: 2.0847 - val_accuracy: 0.2665 - lr: 0.0010
Epoch 2/30
45/45 [==============================] - 11s 252ms/step - loss: 1.7443 - accuracy: 0.4109 - val_loss: 1.8081 - val_accuracy: 0.3610 - lr: 0.0010
Epoch 3/30
45/45 [==============================] - 11s 250ms/step - loss: 1.4503 - accuracy: 0.5259 - val_loss: 1.6380 - val_accuracy: 0.4183 - lr: 0.0010
Epoch 4/30
45/45 [==============================] - 11s 249ms/step - loss: 1.2610 - accuracy: 0.6175 - val_loss: 1.4706 - val_accuracy: 0.4842 - lr: 0.0010
Epoch 5/30
45/45 [==============================] - 11s 254ms/step - loss: 1.1283 - accuracy: 0.6515 - val_loss: 1.3685 - val_accuracy: 0.5186 - lr: 0.0010
Epoch 6/30
45/45 [==============================] - 11s 252ms/step - loss: 1.0248 - accuracy: 0.7083 - val_loss: 1.2998 - val_accuracy: 0.5387 - lr: 0.0010
Epoch 7/30
45/45 [==============================] - 11s 250ms/st

In [75]:
model2.save("basic.h5")

In [81]:
from keras.models import load_model
model_path = Path('basic.h5')

if model_path.exists():
    model2 = load_model(model_path)
    print("Model loaded!")

Model loaded!


In [82]:
for layer in model2.layers[:-20]:
    layer.trainable = False

model2.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6),
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
]

history = model2.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/30
45/45 [==============================] - 14s 261ms/step - loss: 0.2097 - accuracy: 0.9510 - val_loss: 0.5089 - val_accuracy: 0.8309 - lr: 1.0000e-04
Epoch 2/30
45/45 [==============================] - 11s 250ms/step - loss: 0.2117 - accuracy: 0.9546 - val_loss: 0.5058 - val_accuracy: 0.8338 - lr: 1.0000e-04
Epoch 3/30
45/45 [==============================] - 11s 248ms/step - loss: 0.2187 - accuracy: 0.9432 - val_loss: 0.5061 - val_accuracy: 0.8338 - lr: 1.0000e-04
Epoch 4/30
45/45 [==============================] - 11s 252ms/step - loss: 0.2045 - accuracy: 0.9546 - val_loss: 0.5070 - val_accuracy: 0.8309 - lr: 1.0000e-04
Epoch 5/30
45/45 [==============================] - 11s 250ms/step - loss: 0.2103 - accuracy: 0.9553 - val_loss: 0.5089 - val_accuracy: 0.8281 - lr: 1.0000e-04
Epoch 6/30
45/45 [==============================] - 11s 250ms/step - loss: 0.2102 - accuracy: 0.9517 - val_loss: 0.5083 - val_accuracy: 0.8281 - lr: 5.0000e-05
Epoch 7/30
45/45 [======================

## 7. Visualize Training History

In [ ]:
# Plot training history
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

## 8. Evaluate Model
Testing on unseen eval dataset

In [ ]:
# Evaluate model on eval/ folder
eval_datagen = ImageDataGenerator(rescale=1./255)

eval_generator = eval_datagen.flow_from_directory(
    EVAL_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

print('Evaluating model on eval dataset...')
eval_loss, eval_accuracy = model.evaluate(eval_generator)
print(f'\nEval Loss: {eval_loss:.4f}')
print(f'Eval Accuracy: {eval_accuracy:.4f}')

## 9. Confusion Matrix & Classification Report

In [ ]:
# Generate predictions for confusion matrix
eval_generator.reset()
predictions = model.predict(eval_generator, verbose=1)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = eval_generator.classes

# Create confusion matrix
cm = confusion_matrix(true_classes, predicted_classes)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=range(10), yticklabels=range(10))
plt.title('Confusion Matrix - Sign Language Digits')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

# Print classification report
print('\nClassification Report:')
print(classification_report(true_classes, predicted_classes, 
                          target_names=[str(i) for i in range(10)]))